## Experiment Setup

### Random seed / PyTorch / CUDA related

In [1]:
import torch
import numpy as np

# Google Colab-only setup.
use_colab = True

# Is this running on Colab?
import importlib
colab_available = importlib.util.find_spec("google.colab") is not None

if use_colab and colab_available:
    # Mount my Google Drive root folder
    from google.colab import drive
    drive.mount('/content/drive')

    # cd to bayesian-dl-experiments directory
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    !ls

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(682)
np.random.seed(682)

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
    use_pin_memory=True # Faster Host to GPU copies with page-locked memory
else:
    torch_device = torch.device('cpu')
    use_pin_memory=False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/bayesian-dl-experiments
datasets_files			LICENSE     test_results
experiment_comparison.ipynb	README.md
experiment_nn_capacity_1.ipynb	ronald_bdl


### Variable settings

In [0]:
# Dataset to use
dataset_name = 'protein-tertiary-structure'

# Training set size
dataset_train_size = 0.8

# Epochs
n_epochs = 400

# Data batch sizes
n_training_batch = 128

# Set the training/test set sizes
subset_proportions = [0.4, 0.6, 0.8, 1]

# NN hyperparameters
network_hidden_dims = [25, 50, 100]
network_hidden_layers = [1, 3, 5]
network_dropout_rates = [0.005, 0.01, 0.05, 0.1]

# Regularization strengths
regularization_strengths = [0.025, 0.05, 0.075]

# Number of test predictions (for each data point)
prediction_runs = [3, 10, 100, 1000, 3000, 5000, 7000, 10000]


### Training setup

In [0]:
import time
import datetime
import os
from itertools import product

from torch import nn, optim
from torch.utils.data import random_split, DataLoader, RandomSampler
from ronald_bdl import models, datasets

# Mean Squared Error for loss function to minimize
objective = nn.MSELoss()

# Test start time
test_start_time = datetime.datetime.today().strftime('%Y%m%d%H%M')

## Train the network

In [4]:
for subset_prop, hidden_dim, n_hidden, dropout_rate, reg_strength in product(
    subset_proportions,
    network_hidden_dims, network_hidden_layers, network_dropout_rates,
    regularization_strengths,
):
    
    # Create directory to store results for the current test configuration
    test_results_path = os.path.join(
        './test_results',
        'nn_capacity_1',
        dataset_name,
        test_start_time,
        (
            str(subset_prop) 
            + '_' + str(hidden_dim) 
            + '_' + str(n_hidden) 
            + '_' + str(dropout_rate) 
            + '_' + str(reg_strength)),
    )
    
    os.makedirs(test_results_path)
    
    test_results_rmse_mc_path = os.path.join(
        test_results_path,
        "rmse_mc.txt"
    )
    
    test_results_lls_mc_path = os.path.join(
        test_results_path,
        "lls_mc.txt"
    )
    
    test_results_times_mc_path = os.path.join(
        test_results_path,
        "times_mc.txt"
    )

    # Prepare new subset of the original dataset
    subset = datasets.UCIDatasets(
        dataset_name, root_dir='./datasets_files', 
        limit_size=subset_prop, transform=None, download=True)

    # Determine sizes of training and testing set
    train_size = int(dataset_train_size * len(subset))
    test_size = len(subset) - train_size
    
    # Print the size of the subset
    print("subset size = " + str((len(subset), subset.n_features)))
    print("training set size = %d" % train_size)
    print("test set size = %d" % test_size)
    
    train, test = random_split(subset, lengths=[train_size, test_size])

    train_loader = DataLoader(train, batch_size=n_training_batch, pin_memory=use_pin_memory)

    # Prepare network
    network = models.FCNetMCDropout(
        input_dim=subset.n_features, 
        output_dim=subset.n_targets,
        hidden_dim=hidden_dim,
        n_hidden=n_hidden,
        dropout_rate=dropout_rate,
        dropout_type='bernoulli',
    )
    
    # Send the whole model to the selected torch.device
    network.to(torch_device)

    # Print the network structure
    print(network)
    
    # Model to train mode
    network.train()
    
    # Adam optimizer
    # https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam
    # NOTE: Need to set L2 regularization from here
    optimizer = optim.Adam(
        network.parameters(),
        lr=0.01,
        weight_decay=reg_strength, # L2 regularization
    )

    print()

    """
    Training
    """

    print(
        "Starting subset %f, n_hidden %d, hidden_dim %d, dropout_rate %f, reg_strength %f"
        % (subset_prop, n_hidden, hidden_dim, dropout_rate, reg_strength))

    # Record training start time (for this split)
    tic = time.time()

    for epoch in range(n_epochs): # loop over the dataset multiple times

        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, targets = data

            # Store the batch to torch_device's memory
            inputs = inputs.to(torch_device)
            targets = targets.to(torch_device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = network(inputs)

            loss = objective(outputs, targets)
            loss.backward()

            optimizer.step()
            
    # Record training end time
    toc = time.time()

    # Report the final loss
    print("final loss = %f" % (loss.item()))

    """
    Testing
    """

    # Model to eval mode
    #network.eval()

    # Get the test data
    inputs, targets = test.dataset[test.indices]

    # Store the batch to torch_device's memory
    inputs = inputs.to(torch_device)
    targets = targets.to(torch_device)

    for n_predictions in prediction_runs:
        print(str(n_predictions) + " test runs...")

        # Record testing start time (for this split)
        tic_testing = time.time()

        _, mean, var, metrics = network.predict_dist(inputs, n_predictions, y_test=targets, reg_strength=reg_strength)

        # Record testing end time
        toc_testing = time.time()

        # store additional metrics
        if len(metrics) > 0:
            with open(test_results_times_mc_path, 'a+') as times_mc_file:
                times_mc_file.write('%d %f \n' % (n_predictions, toc_testing-tic_testing))

            for key, value in metrics.items():
                print(str(key) + " = " + str(value))

                if key == 'rmse_mc':
                    with open(test_results_rmse_mc_path, 'a+') as rmse_mc_file:
                        rmse_mc_file.write('%d %f \n' % (n_predictions, value))

                elif key == 'test_ll_mc':
                    with open(test_results_lls_mc_path, 'a+') as lls_mc_file:
                        lls_mc_file.write('%d %f \n' % (n_predictions, value))
            print()
            
    # Report the total training time
    print("training time = " + str(toc - tic) + " seconds")
    
    # Report the total testing time
    print("testing time (last run) = " + str(toc_testing - tic_testing) + " seconds")

    print()

Using downloaded and verified file: ./datasets_files/yacht/yacht_hydrodynamics.data
subset size = (123, 6)
training set size = 98
test set size = 25
FCNetMCDropout(
  (input): Sequential(
    (0): Linear(in_features=6, out_features=25, bias=True)
    (1): Dropout(p=0.005, inplace=False)
  )
  (hidden_layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=25, out_features=25, bias=True)
      (1): Dropout(p=0.005, inplace=False)
    )
  )
  (output): Linear(in_features=25, out_features=1, bias=True)
)

Starting subset 0.400000, n_hidden 1, hidden_dim 25, dropout_rate 0.005000, reg_strength 0.025000
final loss = 7.751255
10 test runs...
rmse_mc = tensor(3.1010, device='cuda:0')
rmse_non_mc = tensor(2.9446, device='cuda:0')
test_ll_mc = tensor(-2.8932, device='cuda:0')

100 test runs...
rmse_mc = tensor(2.9338, device='cuda:0')
rmse_non_mc = tensor(2.9446, device='cuda:0')
test_ll_mc = tensor(-2.8790, device='cuda:0')

1000 test runs...
rmse_mc = tensor(2.9392, device='cu